---
# --- Day 19: Monster Messages ---
---

In [41]:
import numpy as np
import itertools

### Input

In [184]:
with open("data/19_input.txt") as f:
    data = [l.strip() for l in f.readlines()]
with open("data/19_input_test.txt") as f:
    data_test = [l.strip() for l in f.readlines()]

In [114]:
def interpret_data(data):
    rules = {}
    rules_to_fill = {}
    for i, l in enumerate(data):
        if l == "":
            break
        parts = l.split(": ")
        k = int(parts[0])
        if parts[1][0] == '"':
            rules[k] = set([str(parts[1][1:-1])])
        else:
            rules_to_fill[k] = [[int(n) for n in p.split(" ")] for p in parts[1].split(" | ")]
    messages = data[i+1:]
    return rules, rules_to_fill, messages   

In [115]:
rules, rules_to_fill, messages = interpret_data(data)

### Part 1: fill rules

In [116]:
def fill_rules(rules, rules_to_fill):
    complete_rules = rules.copy()
    rules_to_fill_int = rules_to_fill.copy()
    available_rules = set(complete_rules.keys())
    print(f"Number of available rules: {len(available_rules)}.")
    print(f"Number of rules still to fill: {len(rules_to_fill)}.")
    while (len(rules_to_fill_int) > 0):
        fillable_rules = [key for key, rule in rules_to_fill_int.items() if len(set([el for comb in rule for el in comb]).difference(available_rules)) == 0]
        if len(fillable_rules) == 0:
            print("Impossible to continue!")
            break
        else:
            for rid in fillable_rules:
                new_rule = [["".join(p) for p in list(itertools.product(*[complete_rules[i] for i in comb]))] for comb in rules_to_fill_int[rid]]
                complete_rules[rid] = set([el for comb in new_rule for el in comb])
                del rules_to_fill_int[rid]
            available_rules = available_rules.union(set(fillable_rules))
            print(f"Number of available rules: {len(available_rules)}.")
            print(f"Number of rules still to fill: {len(rules_to_fill_int)}.")            
    
    return complete_rules, rules_to_fill_int

In [117]:
complete_rules = fill_rules(rules, rules_to_fill)[0]

Number of available rules: 2.
Number of rules still to fill: 127.
Number of available rules: 12.
Number of rules still to fill: 117.
Number of available rules: 41.
Number of rules still to fill: 88.
Number of available rules: 75.
Number of rules still to fill: 54.
Number of available rules: 97.
Number of rules still to fill: 32.
Number of available rules: 112.
Number of rules still to fill: 17.
Number of available rules: 120.
Number of rules still to fill: 9.
Number of available rules: 124.
Number of rules still to fill: 5.
Number of available rules: 126.
Number of rules still to fill: 3.
Number of available rules: 128.
Number of rules still to fill: 1.
Number of available rules: 129.
Number of rules still to fill: 0.


In [118]:
complete_rules[0]

{'aabbbababaaabbaababaabba',
 'abaababbbbabbababbbbbbaa',
 'ababbabbbbabaaabababaaaa',
 'aaaabababbbbbaaabaaaabaa',
 'abbbbabbbbbabbbbbaaaaaba',
 'ababbbaaaabaabaabbbbaaaa',
 'aabababaaaaaaaababaabaaa',
 'aabbbbbaaabbbababbbbbbba',
 'babaaaaabbbbbaaaaabbabba',
 'baaaaaabaabbbaabaaaababb',
 'abbbabbabbbaabaabaaaabab',
 'babbaabbaabbababbbbababa',
 'abbaaabaaabbaababbaababa',
 'aaabbbbbbaaabaabaabbaaab',
 'babaabbbbbbaaaabbbbabbab',
 'ababbbbbababbabbbabbbaaa',
 'aaaaabbbabbaaababbbabbba',
 'aaaababaaabababaabaababa',
 'aaababababbabbaaaabaaaaa',
 'abbababbabaaabbbababbaaa',
 'bbabbbbbabbaabbabbbabaaa',
 'aaaabaabaababbbabbaaabba',
 'bbbaaaaaabbabbaaaababaaa',
 'aabaabaabaaababbbbbbaaba',
 'aaaabbbaabaabbbbbaabaabb',
 'bbaabbaabbabababbbbabaab',
 'bbabbabbabaaabaabbaaabaa',
 'aabbbbabaabbbbaabbbabbba',
 'bbaabbababbaaabbaaabbbaa',
 'aababbaaaabbbabaabaaaaab',
 'aabbbbbbabaabbabbabbbaab',
 'babaaabaaababbbbbaaaaaaa',
 'bbbbbaaaaabbbbaabaaababa',
 'bbabaabaabaaabbbbabbbaaa',
 'baabbbaaabba

In [119]:
sum([m in complete_rules[0] for m in messages])

205

### Part 2: changes of rules 8 and 11

In [120]:
rules_to_fill[8] = [[42], [42, 8]]
rules_to_fill[11] = [[42, 31], [42,11,31]]

In [132]:
partial_rules, remaining_rules = fill_rules(rules, rules_to_fill)

Number of available rules: 2.
Number of rules still to fill: 127.
Number of available rules: 12.
Number of rules still to fill: 117.
Number of available rules: 41.
Number of rules still to fill: 88.
Number of available rules: 75.
Number of rules still to fill: 54.
Number of available rules: 97.
Number of rules still to fill: 32.
Number of available rules: 112.
Number of rules still to fill: 17.
Number of available rules: 120.
Number of rules still to fill: 9.
Number of available rules: 124.
Number of rules still to fill: 5.
Number of available rules: 126.
Number of rules still to fill: 3.
Impossible to continue!


In [133]:
remaining_rules

{8: [[42], [42, 8]], 0: [[8, 11]], 11: [[42, 31], [42, 11, 31]]}

In [134]:
{len(d) for d in partial_rules[42]}

{8}

In [152]:
{len(d) for d in partial_rules[31]}

{8}

In [153]:
{len(m) for m in messages}

{24, 32, 40, 48, 56, 64, 72, 80, 88}

In [227]:
def get_matching_messages(messages, rule42, rule31, msglength=8):
    good_messages = []
    for j, m in enumerate(messages):
        print(f"Checking message {j+1}...", end=" ")
        len_head = int(np.floor(len(m)/msglength/2+1))*msglength
        all_head_matching_42 = np.all([part in rule42 for part in [m[i:i+msglength] for i in range(0,len_head,msglength)]])
        still_good = all_head_matching_42 and (m[-msglength:] in rule31)
        if still_good and False:
            middle = m[len_head:-msglength]
            i = 0
            change = False
            while still_good and (i < len(middle)):
                bite = middle[i:i+msglength]
                if not change:
                    if bite in rule42:
                        pass
                    elif bite in rule31:
                        change = True
                    else:
                        still_good = False
                if change:
                    if bite in rule31:
                        still_good = True
                    else:
                        still_good = False
                i += msglength
        if still_good:
            print("added.")
            good_messages.append(m)
        else:
            print("not added.")
    return good_messages

In [228]:
good_messages =  get_matching_messages(messages, partial_rules[42], partial_rules[31])

Checking message 1... added.
Checking message 2... not added.
Checking message 3... not added.
Checking message 4... added.
Checking message 5... not added.
Checking message 6... added.
Checking message 7... not added.
Checking message 8... added.
Checking message 9... added.
Checking message 10... not added.
Checking message 11... not added.
Checking message 12... added.
Checking message 13... added.
Checking message 14... added.
Checking message 15... added.
Checking message 16... added.
Checking message 17... added.
Checking message 18... added.
Checking message 19... not added.
Checking message 20... not added.
Checking message 21... added.
Checking message 22... added.
Checking message 23... not added.
Checking message 24... not added.
Checking message 25... not added.
Checking message 26... added.
Checking message 27... added.
Checking message 28... added.
Checking message 29... not added.
Checking message 30... added.
Checking message 31... added.
Checking message 32... not adde

In [229]:
len(good_messages)

329

### Test

In [230]:
rules_t, rules_to_fill_t, messages_t = interpret_data(data_test)

In [231]:
complete_rules_t = fill_rules(rules_t, rules_to_fill_t)[0]

Number of available rules: 2.
Number of rules still to fill: 29.
Number of available rules: 10.
Number of rules still to fill: 21.
Number of available rules: 17.
Number of rules still to fill: 14.
Number of available rules: 22.
Number of rules still to fill: 9.
Number of available rules: 26.
Number of rules still to fill: 5.
Number of available rules: 28.
Number of rules still to fill: 3.
Number of available rules: 30.
Number of rules still to fill: 1.
Number of available rules: 31.
Number of rules still to fill: 0.


In [232]:
sum([m in complete_rules_t[0] for m in messages_t])

3

In [233]:
rules_to_fill_t[8] = [[42], [42, 8]]
rules_to_fill_t[11] = [[42, 31], [42,11,31]]

In [234]:
partial_rules_t, remaining_rules_t = fill_rules(rules_t, rules_to_fill_t)

Number of available rules: 2.
Number of rules still to fill: 29.
Number of available rules: 10.
Number of rules still to fill: 21.
Number of available rules: 17.
Number of rules still to fill: 14.
Number of available rules: 22.
Number of rules still to fill: 9.
Number of available rules: 26.
Number of rules still to fill: 5.
Number of available rules: 28.
Number of rules still to fill: 3.
Impossible to continue!


In [237]:
good_messages_t =  get_matching_messages(messages_t, partial_rules_t[42], partial_rules_t[31], msglength=5)

Checking message 1... not added.
Checking message 2... added.
Checking message 3... added.
Checking message 4... added.
Checking message 5... added.
Checking message 6... added.
Checking message 7... added.
Checking message 8... added.
Checking message 9... added.
Checking message 10... added.
Checking message 11... added.
Checking message 12... not added.
Checking message 13... added.
Checking message 14... not added.
Checking message 15... added.


In [238]:
len(good_messages_t)

12